# Release METype Densities

## Imports

In [78]:
import json
import rdflib
import getpass
import pandas as pd
from rdflib import RDF, RDFS, XSD, OWL, URIRef, BNode, SKOS
import pprint
from kgforge.core import KnowledgeGraphForge

## Setup
Get an authentication token

For now, the [Nexus web application](https://bbp.epfl.ch/nexus/web) can be used to get a token. We are looking for other simpler alternatives.

- Step 1: From the opened web page, click on the login button on the right corner and follow the instructions.

![login-ui](./login-ui.png)

- Step 2: At the end you’ll see a token button on the right corner. Click on it to copy the token.

![login-ui](./copy-token.png)


In [79]:
TOKEN = ""

In [80]:
endpoint_staging = "https://staging.nise.bbp.epfl.ch/nexus/v1"
endpoint_prod = "https://bbp.epfl.ch/nexus/v1"
endpoint = endpoint_staging

In [81]:
# To query the metype density nrrd files
forge = KnowledgeGraphForge("../forge-config.yml",
                            token = TOKEN,
                            endpoint = endpoint,
                            bucket = "bbp/atlas")

In [82]:
# To write in the bbp/atlasdatasetrelease project
forge_atlas_release = KnowledgeGraphForge("../forge-config.yml",
                            token = TOKEN,
                            endpoint = endpoint,
                            bucket = "bbp/atlasdatasetrelease")

## Set the atlas release id
These atlas releases can be explored through the atlas web app:

* dev: https://bluebrainatlas.kcpdev.bbp.epfl.ch/atlas
* prod: https://bbp.epfl.ch/atlas


In [83]:

# Prod
Prod_BBP_Mouse_Brain_Atlas_Release = "https://bbp.epfl.ch/neurosciencegraph/data/4906ab85-694f-469d-962f-c0174e901885" 
Prod_Cell_Compostion_Volume_Release_Id = "https://bbp.epfl.ch/neurosciencegraph/data/27652d4d-3a6f-42c7-9833-64396104c445"
Prod_Cell_Compostion_Volume_Release_Version = "v0.3.2"
# Staging
Staging_BBP_Mouse_Brain_Atlas_Release = "https://bbp.epfl.ch/neurosciencegraph/data/brainatlasrelease/c96c71a8-4c0d-4bc1-8a1a-141d9ed6693d" 
Staging_Cell_Compostion_Volume_Release_Id = "https://bbp.epfl.ch/neurosciencegraph/data/celldensities/760b887c-759a-4ded-98e6-5d2712349eb2"
Staging_Cell_Compostion_Volume_Release_Version = 1

atlas_release_id = Staging_BBP_Mouse_Brain_Atlas_Release
cell_compostion_volume_release_id = Staging_Cell_Compostion_Volume_Release_Id
cell_compostion_volume_release_version = Staging_Cell_Compostion_Volume_Release_Version


## Get the atlas release high level metadata

In [84]:
atlas_release = forge.retrieve(atlas_release_id)

In [ ]:
print(atlas_release)

In [85]:
atlas_release._store_metadata["_rev"]

7

## Get the metype density nrrd files

### Get the metype density nrrd files annotated with Mtypes with layers

In [86]:
# Density resources annotated with Mtypes without layers are note released
query = f"""
SELECT DISTINCT ?s
WHERE {{
        ?s a METypeDensity ;
            atlasRelease <{atlas_release_id}>; 
            annotation / hasBody / label ?mtype_label ;
            brainLocation / brainRegion ?brainRegion ;  
            brainLocation / layer ?layer;
            distribution ?distribution .
        ?distribution name ?nrrd_file ;
        contentUrl ?contentUrl .
}}
"""

In [87]:
from tqdm.notebook import tqdm
all_resources_with_layer = forge.sparql(query, limit=1000, debug=False)
resources = [forge.retrieve(id = r.s) for r in tqdm(all_resources_with_layer)]
print(f"{len(resources)} ME-type dentisities with layer found")

  0%|          | 0/243 [00:00<?, ?it/s]

243 metype dentisities found


### Some sanity check of the results of the previous query

In [88]:
# Get all metype densities
all_resources = forge.search({"type":"METypeDensity", "atlasRelease":{"@id":atlas_release_id}}, limit=1000, debug=False)
len(all_resources)

318

In [ ]:
# So 318 - 243 = 75 without layer
# 75 without layers = -3 SAC -1 LAC -3 HAC -2 DAC -4 NGC-DA - 7 (dbc) - 3 (chc) - 10 (nestbc) - 10 (lbc) - 3 (sbc) - 7 (btc) - 6 (bc) - 9 (mc) - 5 (neurogliaform Cell) - 1 (GenericExc) -  1 (GenericInh)
# 1 (GenericExc) and  1 (GenericInh) will be included in the final dataset

In [90]:
mtype_ids_to_exclude_list = ["http://uri.interlex.org/base/ilx_0784350", "http://uri.interlex.org/base/ilx_0107356", 
            "http://uri.interlex.org/base/ilx_0777217", "http://uri.interlex.org/base/ilx_0777215", "http://uri.interlex.org/base/ilx_0777219",
            "http://uri.interlex.org/base/ilx_0777212", "https://bbp.epfl.ch/ontologies/core/mtypes/BipolarCell",
            "http://uri.interlex.org/base/ilx_0107375","http://uri.interlex.org/base/ilx_0777218",
             "https://bbp.epfl.ch/ontologies/core/bmo/GenericExcitatoryNeuronMType",
              "https://bbp.epfl.ch/ontologies/core/bmo/GenericInhibitoryNeuronMType",
              'https://bbp.epfl.ch/ontologies/core/bmo/NGC-DA',
 'https://bbp.epfl.ch/ontologies/core/bmo/DAC',
 'https://bbp.epfl.ch/ontologies/core/bmo/HAC',
 'https://bbp.epfl.ch/ontologies/core/bmo/LAC',
 'https://bbp.epfl.ch/ontologies/core/bmo/SAC']

In [91]:
check_all_resources_without_layer = [r for r in all_resources if not hasattr(r.brainLocation,"layer") ]

all_annotations = [r.annotation for r in check_all_resources_without_layer]
all_annotations = [a  for a_list in all_annotations for a in a_list]
all_annotations_mtype = [a for a in all_annotations if "MType" in a.hasBody.type and a.hasBody.id not in mtype_ids_to_exclude_list]
assert len(all_annotations_mtype) == 0

### Add to the release density for the generic types
* GenericExcitatoryNeuronMType
* GenericExcitatoryNeuronEType
* GenericInhibitoryNeuronMType
* GenericInhibitoryNeuronEType

In [92]:
# Get GenericExcitatoryNeuron
query = f"""

SELECT DISTINCT ?s

WHERE {{
        ?s a METypeDensity ;
            atlasRelease <{atlas_release_id}>; 
            annotation / hasBody <https://bbp.epfl.ch/ontologies/core/bmo/GenericExcitatoryNeuronMType> ;
            annotation / hasBody <https://bbp.epfl.ch/ontologies/core/bmo/GenericExcitatoryNeuronEType> ;
            brainLocation / brainRegion ?brainRegion ;
            distribution ?distribution .
        ?distribution name ?nrrd_file ;
            contentUrl ?contentUrl .
}}
"""
generic_exc_resources = forge.sparql(query, limit=1000, debug=False)
assert len(generic_exc_resources) == 1
generic_exc_resource = forge.retrieve(id=generic_exc_resources[0].s)

In [93]:
# Get GenericInhibitoryNeuron

query = f"""

SELECT DISTINCT ?s

WHERE {{
        ?s a METypeDensity ;
            atlasRelease <{atlas_release_id}>; 
            annotation / hasBody <https://bbp.epfl.ch/ontologies/core/bmo/GenericInhibitoryNeuronMType> ;
            annotation / hasBody <https://bbp.epfl.ch/ontologies/core/bmo/GenericInhibitoryNeuronEType> ;
            brainLocation / brainRegion ?brainRegion ;
            distribution ?distribution .
        ?distribution name ?nrrd_file ;
            contentUrl ?contentUrl .
}}
"""
generic_inh_resources = forge.sparql(query, limit=1000, debug=False)#forge.search({"type":"METypeDensity","atlasRelease":{"@id":atlas_release_id}}, limit=1000, debug=False)
assert len(generic_inh_resources) == 1
generic_inh_resource = forge.retrieve(id=generic_inh_resources[0].s)

In [94]:
# Add the generic densities to the release set
resources.append(generic_exc_resource)
resources.append(generic_inh_resource)
print(f"{len(resources)} metype densities will be released, including generic ones")

245 metype dentisities will be released including generic ones


### Display the densities to release as dataframe

In [95]:
# Collect metadata as pandas dataframe 
reshaped_resources = forge.reshape(resources, keep=["id","_store_metadata._self","type", "annotation.hasBody.id", "annotation.hasBody.label",
                     "brainLocation.brainRegion.id", "brainLocation.brainRegion.label", "distribution.atLocation.location", "_store_metadata._rev"])
df = forge.as_dataframe(reshaped_resources, store_metadata = True, nesting=".")

df["mtype"] = df.apply(lambda row: (row.annotation[0]["hasBody"]["label"],row.annotation[0]["hasBody"]["id"]) , axis=1) 
df["etype"] = df.apply(lambda row: (row.annotation[1]["hasBody"]["label"],row.annotation[1]["hasBody"]["id"]) , axis=1) 
type_column = df.pop('type')
mtype_column = df.pop('mtype')
etype_column = df.pop('etype')

df.insert(0, 'type', type_column)
df.insert(1, 'mtype', mtype_column)
df.insert(2, 'etype', etype_column)

df.drop(columns="annotation")

df.head(100)

,type,mtype,etype,id,annotation,brainLocation.brainRegion.id,brainLocation.brainRegion.label,distribution.atLocation,_constrainedBy,_createdAt,_createdBy,_deprecated,_incoming,_outgoing,_project,_rev,_schemaProject,_self,_updatedAt,_updatedBy
0,"[NeuronDensity, VolumetricDataLayer, CellDensi...","(L6_TPC:A, http://uri.interlex.org/base/ilx_03...","(cADpyr, http://bbp.epfl.ch/neurosciencegraph/...",https://bbp.epfl.ch/neurosciencegraph/data/den...,[{'hasBody': {'id': 'http://uri.interlex.org/b...,http://api.brain-map.org/api/v2/data/Structure...,root,NaN,https://neuroshapes.org/dash/volumetricdatalayer,2022-10-24T17:44:55.865Z,https://staging.nise.bbp.epfl.ch/nexus/v1/real...,False,https://staging.nise.bbp.epfl.ch/nexus/v1/reso...,https://staging.nise.bbp.epfl.ch/nexus/v1/reso...,https://staging.nise.bbp.epfl.ch/nexus/v1/proj...,10,https://staging.nise.bbp.epfl.ch/nexus/v1/proj...,https://staging.nise.bbp.epfl.ch/nexus/v1/reso...,2022-11-11T08:21:05.839Z,https://staging.nise.bbp.epfl.ch/nexus/v1/real...
1,"[NeuronDensity, VolumetricDataLayer, CellDensi...","(L3_TPC:B, http://uri.interlex.org/base/ilx_03...","(cADpyr, http://bbp.epfl.ch/neurosciencegraph/...",https://bbp.epfl.ch/neurosciencegraph/data/den...,[{'hasBody': {'id': 'http://uri.interlex.org/b...,http://api.brain-map.org/api/v2/data/Structure...,root,NaN,https://neuroshapes.org/dash/volumetricdatalayer,2022-10-24T17:44:56.714Z,https://staging.nise.bbp.epfl.ch/nexus/v1/real...,False,https://staging.nise.bbp.epfl.ch/nexus/v1/reso...,https://staging.nise.bbp.epfl.ch/nexus/v1/reso...,https://staging.nise.bbp.epfl.ch/nexus/v1/proj...,10,https://staging.nise.bbp.epfl.ch/nexus/v1/proj...,https://staging.nise.bbp.epfl.ch/nexus/v1/reso...,2022-11-11T08:21:07.047Z,https://staging.nise.bbp.epfl.ch/nexus/v1/real...
2,"[NeuronDensity, VolumetricDataLayer, CellDensi...","(L5_TPC:A, http://uri.interlex.org/base/ilx_03...","(cADpyr, http://bbp.epfl.ch/neurosciencegraph/...",https://bbp.epfl.ch/neurosciencegraph/data/den...,[{'hasBody': {'id': 'http://uri.interlex.org/b...,http://api.brain-map.org/api/v2/data/Structure...,root,NaN,https://neuroshapes.org/dash/volumetricdatalayer,2022-10-24T17:44:57.604Z,https://staging.nise.bbp.epfl.ch/nexus/v1/real...,False,https://staging.nise.bbp.epfl.ch/nexus/v1/reso...,https://staging.nise.bbp.epfl.ch/nexus/v1/reso...,https://staging.nise.bbp.epfl.ch/nexus/v1/proj...,10,https://staging.nise.bbp.epfl.ch/nexus/v1/proj...,https://staging.nise.bbp.epfl.ch/nexus/v1/reso...,2022-11-11T08:21:07.983Z,https://staging.nise.bbp.epfl.ch/nexus/v1/real...
3,"[NeuronDensity, VolumetricDataLayer, CellDensi...","(L2_TPC:B, http://uri.interlex.org/base/ilx_03...","(cADpyr, http://bbp.epfl.ch/neurosciencegraph/...",https://bbp.epfl.ch/neurosciencegraph/data/den...,[{'hasBody': {'id': 'http://uri.interlex.org/b...,http://api.brain-map.org/api/v2/data/Structure...,root,NaN,https://neuroshapes.org/dash/volumetricdatalayer,2022-10-24T17:44:58.461Z,https://staging.nise.bbp.epfl.ch/nexus/v1/real...,False,https://staging.nise.bbp.epfl.ch/nexus/v1/reso...,https://staging.nise.bbp.epfl.ch/nexus/v1/reso...,https://staging.nise.bbp.epfl.ch/nexus/v1/proj...,10,https://staging.nise.bbp.epfl.ch/nexus/v1/proj...,https://staging.nise.bbp.epfl.ch/nexus/v1/reso...,2022-11-11T08:21:09.023Z,https://staging.nise.bbp.epfl.ch/nexus/v1/real...
4,"[NeuronDensity, VolumetricDataLayer, CellDensi...","(L3_TPC:A, http://uri.interlex.org/base/ilx_03...","(cADpyr, http://bbp.epfl.ch/neurosciencegraph/...",https://bbp.epfl.ch/neurosciencegraph/data/den...,[{'hasBody': {'id': 'http://uri.interlex.org/b...,http://api.brain-map.org/api/v2/data/Structure...,root,NaN,https://neuroshapes.org/dash/volumetricdatalayer,2022-10-24T17:44:59.301Z,https://staging.nise.bbp.epfl.ch/nexus/v1/real...,False,https://staging.nise.bbp.epfl.ch/nexus/v1/reso...,https://staging.nise.bbp.epfl.ch/nexus/v1/reso...,https://staging.nise.bbp.epfl.ch/nexus/v1/proj...,10,https://staging.nise.bbp.epfl.ch/nexus/v1/proj...,https:/

### Locally download the density files

In [97]:
forge.download(resources, path="./test", follow="distribution.contentUrl", cross_bucket=True, overwrite=True)

## Release the densities as CellCompostionVolume entity

### Prepare the density release as a json file to be attached as a distribution

In [101]:
local_file_name="./cellcompositionvolume_release_dataset.json"

In [102]:
grouped_by_metype = {}

metype_annotations = [(a.hasBody for a in r.annotation) for r in resources] 
etype_annotations = [a.hasBody for r in resources for a in r.annotation if "ETypeAnnotation" in a.type]

mtype_to_etype = {}
for i, metype_annotation_gen in enumerate(metype_annotations):
    metype_annotation_gen_list = list(metype_annotation_gen)

    if "MType" in metype_annotation_gen_list[0].type:
        if metype_annotation_gen_list[0].id not in mtype_to_etype:
            mtype_to_etype[metype_annotation_gen_list[0].id] = {"label":metype_annotation_gen_list[0].label}
        if "EType" in metype_annotation_gen_list[1].type and metype_annotation_gen_list[1].id not in mtype_to_etype[metype_annotation_gen_list[0].id]:
            mtype_to_etype[metype_annotation_gen_list[0].id][metype_annotation_gen_list[1].id]={"label":metype_annotation_gen_list[1].label}
        if resources[i].id not in mtype_to_etype[metype_annotation_gen_list[0].id][metype_annotation_gen_list[1].id]:
            mtype_to_etype[metype_annotation_gen_list[0].id][metype_annotation_gen_list[1].id][resources[i].id]={"type":resources[i].type, "_rev":resources[i]._store_metadata._rev}

for k, v in mtype_to_etype.items():
    if "hasPart" not in grouped_by_metype:
        grouped_by_metype["hasPart"] = []
    k_content ={"@id":k, "label":v["label"], "about":["https://neuroshapes.org/MType"]}    
    for kv, vv in v.items():
        if  kv!="label":
            if 'hasPart' not in k_content:
                k_content["hasPart"] = []
            kv_content = {"@id":kv, "label":vv["label"], "about":["https://neuroshapes.org/EType"]}
            for kvv, vvv in vv.items():
                if  kvv!="label":
                    if 'hasPart' not in kv_content:
                        kv_content["hasPart"] = []
                    kv_content["hasPart"].append({"@id":kvv, "@type":vvv["type"], "_rev":vvv["_rev"]})
                    k_content["hasPart"].append(kv_content)
    grouped_by_metype["hasPart"].append(k_content)

with open(local_file_name, "w") as f:
    json.dump(grouped_by_metype, f)


### Create and register a new CellCompostionVolume entity
* The type AtlasDatasetRelease will be kept for backward compatiblity

In [105]:
## Create AtlasDatasetRelease
from kgforge.core import Resource
from kgforge.specializations.resources import Dataset

cell_compostion_volume_release = Dataset(forge_atlas_release, type=["Dataset","AtlasDatasetRelease", "CellCompositionVolume"])

cell_compostion_volume_release.about = ["https://bbp.epfl.ch/ontologies/core/bmo/METypeDensity"]

cell_compostion_volume_release.atlasRelease = Resource(id=atlas_release.id, type=atlas_release.type, _rev=atlas_release._store_metadata._rev)

brainRegion = Resource(id="http://api.brain-map.org/api/v2/data/Structure/997", label="root")

cell_compostion_volume_release.brainLocation= Resource(type="BrainLocation", brainRegion=brainRegion)

cell_compostion_volume_release.atlasSpatialReferenceSystem = Resource(id=atlas_release.spatialReferenceSystem.id, 
                                                            type=atlas_release.spatialReferenceSystem.type,
                                                             _rev=atlas_release._store_metadata._rev)

subject = Resource.from_json({
    "@type": "Subject",
    "species": {
      "@id": "http://purl.obolibrary.org/obo/NCBITaxon_10090",
      "label": "Mus musculus"
    }
  })

cell_compostion_volume_release.subject = subject

# 
contribution = [
  Resource.from_json({ # Always add EPFL
    "@type": "Contribution",
    "agent": {
      "@id": "https://www.grid.ac/institutes/grid.5333.6",
      "@type": [
        "Agent",
        "Organization"
      ],
      "name": "École Polytechnique Fédérale de Lausanne"
    }
  }),
  Resource.from_json({ # To be changed. The connected user can also be added (info from the token)
    "@type": "Contribution",
    "agent": {
      "@id": "https://bbp.epfl.ch/nexus/v1/realms/bbp/users/sy",
      "@type": [
        "Agent",
        "Person"
      ],
      "name": "Mohameth François Sy"
    }
  })
  
  ]
cell_compostion_volume_release.contribution = contribution
cell_compostion_volume_release.name = "Transplanted whole brain mouse METypeDensities generated from the atlas pipeline grouped by MType and EType"

cell_compostion_volume_release.description = "Transplanted whole brain mouse METypeDensities generated from the atlas pipeline grouped by MType and EType"

In [106]:
print(cell_compostion_volume_release)

{
    type:
    [
        Dataset
        AtlasDatasetRelease
        CellCompositionVolume
    ]
    about:
    [
        https://bbp.epfl.ch/ontologies/core/bmo/METypeDensity
    ]
    atlasRelease:
    {
        id: https://bbp.epfl.ch/neurosciencegraph/data/brainatlasrelease/c96c71a8-4c0d-4bc1-8a1a-141d9ed6693d
        type:
        [
            AtlasRelease
            BrainAtlasRelease
            Entity
        ]
        _rev: 7
    }
    atlasSpatialReferenceSystem:
    {
        id: https://bbp.epfl.ch/neurosciencegraph/data/allen_ccfv3_spatial_reference_system
        type: AtlasSpatialReferenceSystem
        _rev: 7
    }
    brainLocation:
    {
        type: BrainLocation
        brainRegion:
        {
            id: http://api.brain-map.org/api/v2/data/Structure/997
            label: root
        }
    }
    contribution:
    [
        {
            @type: Contribution
            agent:
            {
                @id: https://www.grid.ac/institutes/grid.5333.6
    

In [107]:
cell_compostion_volume_release.distribution = forge_atlas_release.attach(local_file_name,
                                                                content_type="application/json")

In [109]:
schema_id = forge_atlas_release._model.schema_id(type="CellCompositionVolume")
schema_id

'https://neuroshapes.org/dash/cellcompositionvolume'

In [110]:
forge_atlas_release.register(cell_compostion_volume_release, schema_id=schema_id)

<action> _register_one
<succeeded> True


In [111]:
# Address in Nexus Fusion
cell_compostion_volume_release._store_metadata._self

'https://staging.nise.bbp.epfl.ch/nexus/v1/resources/bbp/atlasdatasetrelease/datashapes:cellcompositionvolume/56f57d4f-ca64-40cc-938c-5017ef4f4091'

### Update an existing CellCompostionVolume entity


In [112]:
cell_compostion_volume_release = forge.retrieve(cell_compostion_volume_release_id,
                                                version = cell_compostion_volume_release_version,
                                                cross_bucket=True)

In [113]:
print(cell_compostion_volume_release)

{
    context: https://bbp.neuroshapes.org
    id: https://bbp.epfl.ch/neurosciencegraph/data/celldensities/760b887c-759a-4ded-98e6-5d2712349eb2
    type:
    [
        CellCompositionVolume
        Entity
    ]
    about:
    [
        nsg:Neuron
        nsg:Glia
    ]
    atlasRelease:
    {
        id: https://bbp.epfl.ch/neurosciencegraph/data/831a626a-c0ae-4691-8ce8-cfb7491345d9
        type:
        [
            AtlasRelease
            BrainAtlasRelease
        ]
        _rev: 2
    }
    brainLocation:
    {
        type: BrainLocation
        brainRegion:
        {
            id: http://api.brain-map.org/api/v2/data/Structure/997
            label: Whole mouse brain
        }
    }
    contribution:
    {
        type: Contribution
        agent:
        {
            id: https://www.grid.ac/institutes/grid.5333.6
            type:
            [
                Agent
                Organization
            ]
        }
    }
    description: The cell composition METypeDensit

In [114]:
cell_compostion_volume_release.distribution = forge_atlas_release.attach(local_file_name,
                                                                content_type="application/json")

In [115]:
forge_atlas_release.update(cell_compostion_volume_release)

<action> _update_one
<succeeded> True


### Tag the CellCompostionVolume entity


In [116]:
new_version = "v0.3.2" # add a new version text here (use semantic versioning)

In [117]:
forge_atlas_release.tag(cell_compostion_volume_release, value=new_version)

<action> _tag_one
<succeeded> True
